In [1]:
import pandas as pd

In [2]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
# exp004
import dataclasses
import torch
import pandas as pd
from sklearn.model_selection import KFold
from transformers import AutoModel, AutoTokenizer, AutoModelForMultipleChoice
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from datasets import Dataset
from typing import Optional, Union

import transformers
import wandb
from datetime import datetime as dt
import os
import numpy as np
import tqdm

import logging 
from logging import Logger



@dataclasses.dataclass
class BertConfig:
    
    experiment_name: str
    dataset_dir: str
    
    debug: bool = False

    lr: float = 1e-5
    model_name: str = "OpenAssistant/reward-model-deberta-v3-large-v2"
    num_context: int = 3
    max_length: int = 512
    batch_size: int = 2
    epochs: int = 10
    iters_to_accumlate: int = 8
    weight_decay: float = 0.01
    warmup_ratio: float = 0.1
    
    freeze_embeddings: bool = True
    freeze_layers: int = 18
    reinitialize_layers: int = 0
    
    assume_completely_retrieved: bool = False
    n_samples: int = None
    steps: int = 100
    
    lora_r: float = 2
    lora_alpha: float = 4
    lora_dropout: float = 0.1
    use_peft: bool = False

def get_logger(
    output_dir: str,
):
    """
    logger を作成する. formatter は "%Y-%m-%d %H:%M:%S" で作成する.
    """
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    
    # formatter
    formatter = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
    
    # handler
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    handler = logging.FileHandler(f"{output_dir}/log.txt", "w")
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger


@dataclasses.dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch


        
def preprocess_df(df, config):
    df["context"] = ""
    for i in range(config.num_context):
        df["context"] += df[f"searched_wiki_id_{i}"].astype(str) + "\n\n"
    
    for col in ["A", "B", "C", "D", "E"]:
        df[col] = df[col].fillna("")
    return df[["prompt", "context", "A", "B", "C", "D", "E", "answer"]]


def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

def compute_metrics(p):
    predictions = p.predictions.tolist()
    labels = p.label_ids.tolist()
    return {"map@3": map_at_3(predictions, labels)}


In [6]:
logger = get_logger(output_dir="")
logger.info("load data")

2023-09-26 06:26:31,059 INFO load data


In [8]:
def calc_map3(
    model_name,
    max_length,
    num_content,
):
    df_test = pd.read_parquet("../output/context_pipeline/stage1/exp009.py/20230925104920_gte-base_wikiall_without_sep_targetprompt_and_choice_without_sep_token_length120_stride_sentence4_drop_categoryTrue_all/valid.parquet")
    option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
    label = df_test["answer"].map(option_to_index).values
    logger = get_logger(output_dir="")
    logger.info("load data")
    
    config = BertConfig(
        debug=False,
        batch_size=1,
        experiment_name=f"",
        dataset_dir="",
        model_name=model_name,
        max_length=max_length,
        num_context=num_content,
    )
    df_test = preprocess_df(df_test, config)
    df_test = df_test[df_test["answer"].isin(["A", "B", "C", "D", "E"])]
    
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
    index_to_option = {v: k for k,v in option_to_index.items()}
    def preprocess(example):
        first_sentence = [ "[CLS] " + example['context'] ] * 5
        second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
        tokenized_example = tokenizer(first_sentence, second_sentences, truncation="only_first", 
                                      max_length=config.max_length, add_special_tokens=False)
        tokenized_example['label'] = option_to_index[example['answer']]
        return tokenized_example

    test_dataset = Dataset.from_pandas(df_test)
    tokenized_test_dataset = test_dataset.map(preprocess, remove_columns=["prompt", "context", "A", "B", "C", "D", "E", "answer", "__index_level_0__"])

    data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
    test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
    
    model = AutoModelForMultipleChoice.from_pretrained(config.model_name).cuda()
    
    test_predictions = []
    for batch in tqdm.tqdm(test_dataloader):
        with torch.autocast("cuda"):
            for k in batch.keys():
                batch[k] = batch[k].cuda()
            with torch.no_grad():
                outputs = model(**batch)
            test_predictions.append(outputs.logits.cpu().detach())

    test_predictions = torch.cat(test_predictions)
    test_predictions = test_predictions.numpy()
    
    print(map_at_3(test_predictions, label))
    return {
        "model_name": model_name,
        "max_length": max_length,
        "num_content": num_content,
        "map3": map_at_3(test_predictions, label),
        "predictions": test_predictions
    }

In [9]:
ret = []

In [10]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/",
    max_length=256,
    num_content=2,
))

2023-09-26 06:26:36,985 INFO load data
2023-09-26 06:26:36,985 INFO load data


  0%|          | 0/4200 [00:00<?, ?ex/s]

100%|███████████████████████████████████████| 4200/4200 [03:35<00:00, 19.45it/s]


0.8719444444444469


In [11]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/",
    max_length=384,
    num_content=3,
))

2023-09-26 06:42:17,023 INFO load data
2023-09-26 06:42:17,023 INFO load data
2023-09-26 06:42:17,023 INFO load data


  0%|          | 0/4200 [00:00<?, ?ex/s]

100%|███████████████████████████████████████| 4200/4200 [03:39<00:00, 19.14it/s]


0.8781349206349229


In [12]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/",
    max_length=512,
    num_content=4,
))

2023-09-26 06:46:27,481 INFO load data
2023-09-26 06:46:27,481 INFO load data
2023-09-26 06:46:27,481 INFO load data
2023-09-26 06:46:27,481 INFO load data


  0%|          | 0/4200 [00:00<?, ?ex/s]

100%|███████████████████████████████████████| 4200/4200 [04:36<00:00, 15.21it/s]


0.8803174603174629


In [13]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/",
    max_length=768,
    num_content=7,
))

2023-09-26 06:51:37,665 INFO load data
2023-09-26 06:51:37,665 INFO load data
2023-09-26 06:51:37,665 INFO load data
2023-09-26 06:51:37,665 INFO load data
2023-09-26 06:51:37,665 INFO load data


  0%|          | 0/4200 [00:00<?, ?ex/s]

100%|███████████████████████████████████████| 4200/4200 [09:32<00:00,  7.33it/s]


0.8750793650793676


In [15]:
df = pd.DataFrame(ret)

In [16]:
df

,model_name,max_length,num_content,map3,predictions
0,../output/stage2/exp005.py/20230925235801_new_...,256,2,0.871944,"[[-11.45, -9.97, -6.613, -8.914, -10.195], [-8..."
1,../output/stage2/exp005.py/20230925235801_new_...,384,3,0.878135,"[[-11.26, -10.39, -5.52, -8.484, -9.77], [-10...."
2,../output/stage2/exp005.py/20230925235801_new_...,512,4,0.880317,"[[-10.94, -10.12, -4.906, -7.586, -9.38], [-10..."
3,../output/stage2/exp005.py/20230925235801_new_...,768,7,0.875079,"[[-10.52, -9.93, -3.178, -6.89, -8.91], [-9.05..."


In [19]:
df_test = pd.read_parquet("../output/context_pipeline/stage1/exp009.py/20230925104920_gte-base_wikiall_without_sep_targetprompt_and_choice_without_sep_token_length120_stride_sentence4_drop_categoryTrue_all/valid.parquet")
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
label = df_test["answer"].map(option_to_index).values


In [20]:
ret2 = []
for i in range(len(df)):
    series = df.iloc[i]
    df_test_ = df_test.copy()
    df_test_["pred"] = [x for x in series["predictions"]]
    
    for source, w_df in df_test_.groupby("source"):
        print(source)
        option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
        label = w_df["answer"].map(option_to_index).values
        test_predictions = np.stack(w_df["pred"].values)
        ret2.append({
            "model_name": series["model_name"],
            "max_length": series["max_length"],
            "num_content": series["num_content"],
            "source": source,
            "map3": map_at_3(test_predictions, label)
        })

1
11
2
20230924083304_gpt5
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
20230924083304_gpt5
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
20230924083304_gpt5
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
20230924083304_gpt5
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet


In [21]:
df_ret2 = pd.DataFrame(ret2)

In [22]:
df_ret2.sort_values(["model_name", "source", "max_length"])

,model_name,max_length,num_content,source,map3
0,../output/stage2/exp005.py/20230925235801_new_...,256,2,1,0.868889
14,../output/stage2/exp005.py/20230925235801_new_...,384,3,1,0.875556
28,../output/stage2/exp005.py/20230925235801_new_...,512,4,1,0.873889
42,../output/stage2/exp005.py/20230925235801_new_...,768,7,1,0.872222
1,../output/stage2/exp005.py/20230925235801_new_...,256,2,11,0.941667
15,../output/stage2/exp005.py/20230925235801_new_...,384,3,11,0.942778
29,../output/stage2/exp005.py/20230925235801_new_...,512,4,11,0.951111
43,../output/stage2/exp005.py/20230925235801_new_...,768,7,11,0.952778
2,../output/stage2/exp005.py/20230925235801_new_...,256,2,2,0.892778
16,../output/stage2/exp005.py/20230925235801_new_...,384,3,2,0.903889


In [23]:
df.to_pickle("20230926_ret.pickle")
df_ret2.to_pickle("20230926_ret2.pickle")

In [24]:
df_ret2.pivot_table(columns=["model_name", "source"], index="max_length", values="map3")

model_name ../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/  \
source                                                                                                    1   
max_length                                                                                                    
256                                                  0.868889                                                 
384                                                  0.875556                                                 
512                                                  0.873889                                                 
768                                                  0.872222                                                 

model_name                                                              \
source            11         2 20230924083304_gpt5         3         4   
max_length                                                               
256         0.941667  0.892778            0.789444  0.860000  0.880556   
384         0.942778  0.903889            0.798333  0.861111  0.887222   
512         0.951111  0.906111            0.787778  0.852778  0.901667   
768         0.952778  0.904444            0.789444  0.840556  0.882778   

model_name                                                    \
source             5         6         7         8         9   
max_length                                                     
256         0.936667  0.883333  0.880000  0.836667  0.795556   
384         0.932222  0.895556  0.884444  0.846667  0.815000   
512         0.938333  0.889444  0.897222  0.848333  0.828889   
768         0.918889  0.876111  0.885000  0.848889  0.827222   

model_name                                \
source     additional_data/MMLU/test.csv   
max_length                                 
256                             0.741667   
384                             0.745556   
512                             0.742778   
768                             0.755000   

model_name                                                                                  \
source     additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv   
max_length                                                                                   
256                                                  0.954444                                
384                                                  0.961111                                
512                                                  0.960000                                
768                                                  0.951111                                

model_name                                         
source     additional_data/ScienceQA/test.parquet  
max_length                                         
256                                      0.945556  
384                                      0.944444  
512                                      0.946111  
768                                      0.946667

In [25]:
df_ret2.pivot_table(index=["model_name", "max_length"], columns="source", values="map3")

source                                                                1  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.868889   
                                                   384         0.875556   
                                                   512         0.873889   
                                                   768         0.872222   

source                                                               11  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.941667   
                                                   384         0.942778   
                                                   512         0.951111   
                                                   768         0.952778   

source                                                                2  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.892778   
                                                   384         0.903889   
                                                   512         0.906111   
                                                   768         0.904444   

source                                                         20230924083304_gpt5  \
model_name                                         max_length                        
../output/stage2/exp005.py/20230925235801_new_d... 256                    0.789444   
                                                   384                    0.798333   
                                                   512                    0.787778   
                                                   768                    0.789444   

source                                                                3  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.860000   
                                                   384         0.861111   
                                                   512         0.852778   
                                                   768         0.840556   

source                                                                4  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.880556   
                                                   384         0.887222   
                                                   512         0.901667   
                                                   768         0.882778   

source                                                                5  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.936667   
                                                   384         0.932222   
                                                   512         0.938333   
                                                   768         0.918889   

source                                                                6  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.883333   
                                                   384         0.895556   
                                                   512         0.889444   
                                                   768         0.876111   

source                                                                7  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.880000   
                                                   384         0.884444   
                          

In [26]:
df_ret2.pivot_table(index=["model_name", "max_length"], columns="source", values="map3")

source                                                                1  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.868889   
                                                   384         0.875556   
                                                   512         0.873889   
                                                   768         0.872222   

source                                                               11  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.941667   
                                                   384         0.942778   
                                                   512         0.951111   
                                                   768         0.952778   

source                                                                2  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.892778   
                                                   384         0.903889   
                                                   512         0.906111   
                                                   768         0.904444   

source                                                         20230924083304_gpt5  \
model_name                                         max_length                        
../output/stage2/exp005.py/20230925235801_new_d... 256                    0.789444   
                                                   384                    0.798333   
                                                   512                    0.787778   
                                                   768                    0.789444   

source                                                                3  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.860000   
                                                   384         0.861111   
                                                   512         0.852778   
                                                   768         0.840556   

source                                                                4  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.880556   
                                                   384         0.887222   
                                                   512         0.901667   
                                                   768         0.882778   

source                                                                5  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.936667   
                                                   384         0.932222   
                                                   512         0.938333   
                                                   768         0.918889   

source                                                                6  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.883333   
                                                   384         0.895556   
                                                   512         0.889444   
                                                   768         0.876111   

source                                                                7  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.880000   
                                                   384         0.884444   
                          

In [27]:
df_ret2[df_ret2["source"].isin(["1", "2", "3", "4", "5", "6", "7", "8"])].groupby(["model_name", "max_length"])["map3"].mean()

model_name                                                                                        max_length
../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/  256           0.879861
                                                                                                  384           0.885833
                                                                                                  512           0.888472
                                                                                                  768           0.878611
Name: map3, dtype: float64

In [55]:
df_ret2[df_ret2["source"].isin(["1", "2", "3", "4", "5", "6", "7", "8"])].groupby(["model_name", "max_length"])["map3"].mean()

model_name                                                                     max_length
../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/  256           0.859722
                                                                               384           0.864931
                                                                               512           0.862153
                                                                               768           0.863750
../output/stage2/exp005.py/20230924021914_new_data_all300val_maxlen384/fold0/  384           0.867014
                                                                               512           0.865417
                                                                               768           0.863264
../output/stage2/exp005.py/20230924084819_new_data_all300val_maxlen512/fold0/  512           0.867222
                                                                               768           0

In [56]:
df_ret2.groupby(["model_name", "max_length"])["map3"].mean()

model_name                                                                     max_length
../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/  256           0.867009
                                                                               384           0.869444
                                                                               512           0.867863
                                                                               768           0.869103
../output/stage2/exp005.py/20230924021914_new_data_all300val_maxlen384/fold0/  384           0.871111
                                                                               512           0.869573
                                                                               768           0.869060
../output/stage2/exp005.py/20230924084819_new_data_all300val_maxlen512/fold0/  512           0.872692
                                                                               768           0

In [72]:
df_test[df_test["A"].apply(len) < 30]["source"].value_counts()

11                                                                                 273
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv    261
9                                                                                  215
additional_data/ScienceQA/test.parquet                                             184
additional_data/MMLU/test.csv                                                      180
8                                                                                  143
7                                                                                  132
4                                                                                  100
3                                                                                   91
2                                                                                   69
5                                                                                   67
1                                          

In [69]:
ret3 = []
for i in range(len(df)):
    series = df.iloc[i]
    df_test_ = df_test.copy()
    df_test_["pred"] = [x for x in series["predictions"]]
    df_test_ = df_test_[df_test_["A"].apply(len) < 20]
    
    for source, w_df in df_test_.groupby("source"):
        print(source)
        option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
        label = w_df["answer"].map(option_to_index).values
        test_predictions = np.stack(w_df["pred"].values)
        ret3.append({
            "model_name": series["model_name"],
            "max_length": series["max_length"],
            "num_content": series["num_content"],
            "source": source,
            "map3": map_at_3(test_predictions, label)
        })

1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/Sci

In [70]:
df_ret3 = pd.DataFrame(ret3)

In [71]:
df_ret3.pivot_table(index=["model_name", "max_length"], columns="source", values="map3")

source                                                                1  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.913194   
                                                   384         0.916667   
                                                   512         0.906250   
                                                   768         0.909722   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.909722   
                                                   512         0.899306   
                                                   768         0.920139   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.913194   
                                                   768         0.909722   

source                                                               11  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.959302   
                                                   384         0.958010   
                                                   512         0.958656   
                                                   768         0.965116   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.955426   
                                                   512         0.955426   
                                                   768         0.965116   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.966408   
                                                   768         0.967054   

source                                                                2  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.833333   
                                                   384         0.884615   
                                                   512         0.852564   
                                                   768         0.868590   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.846154   
                                                   512         0.865385   
                                                   768         0.875000   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.858974   
                                                   768         0.865385   

source                                                                3  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.813853   
                                                   384         0.831169   
                                                   512         0.826840   
                                                   768         0.846320   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.813853   
                                                   512         0.820346   
                                                   768         0.820346   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.826840   
                                                   768         0.865801   

source                                                                4  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.897287   
                                                   384         0.918605   
                                                   512         0.916667   
                                                   768         0.922481   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.918605   
                                                   512         0.926357   
                                                   768         0.926357   
../output/stage2/exp0